In [20]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from utils.studies import bongaarts_study
from utils.utils import bongaarts_matrix, mean_absolute_error, plot_matrix

In [9]:
def objective_builder_factory(a_range, b_range, beta_range, gamma_range, lin=True):
    """
    Returns a function that constructs an objective function for Optuna with specified intervals or fixed values.

    Args:
        a_range (tuple or float): (min, max) range for `a` or a fixed value.
        b_range (tuple or float): (min, max) range for `b` or a fixed value.
        beta_range (tuple or float): (min, max) range for `beta` or a fixed value.
        gamma_range (tuple or float): (min, max) range for `gamma` or a fixed value.
        lin (bool): Linear or not (default is True).

    Returns:
        function: A function that takes weights, ages, years and returns an objective function.
    """

    def create_objective(weights, ages, years):
        # Enforce intervals (tuples) for each parameter or fixed values
        if not (isinstance(a_range, tuple) and len(a_range) == 2 or isinstance(a_range, (float, int))):
            raise ValueError("`a_range` must be a tuple of (min, max) or a fixed value.")
        if not (isinstance(b_range, tuple) and len(b_range) == 2 or isinstance(b_range, (float, int))):
            raise ValueError("`b_range` must be a tuple of (min, max) or a fixed value.")
        if not (isinstance(beta_range, tuple) and len(beta_range) == 2 or isinstance(beta_range, (float, int))):
            raise ValueError("`beta_range` must be a tuple of (min, max) or a fixed value.")
        if not (isinstance(gamma_range, tuple) and len(gamma_range) == 2 or isinstance(gamma_range, (float, int))):
            raise ValueError("`gamma_range` must be a tuple of (min, max) or a fixed value.")

        # Define the actual objective function using the fixed or interval values
        def objective_builder_fixed_intervals(weights, ages, years, a_range, b_range, beta_range, gamma_range, lin=True):
            years = np.array(years)

            def objective(trial):
                # Determine if we use a fixed value or a range
                if isinstance(a_range, tuple):
                    a = trial.suggest_float("a", *a_range)
                else:
                    a = a_range  # fixed value

                if isinstance(b_range, tuple):
                    b = trial.suggest_float("b", *b_range)
                else:
                    b = b_range  # fixed value

                if isinstance(beta_range, tuple):
                    beta = trial.suggest_float("beta", *beta_range, log=True)
                else:
                    beta = beta_range  # fixed value

                if isinstance(gamma_range, tuple):
                    gamma = trial.suggest_float("gamma", *gamma_range, log=True)
                else:
                    gamma = gamma_range  # fixed value

                alpha = np.exp(a * years + b)

                try:
                    QBongaarts = bongaarts_matrix(
                        a, b, beta, -gamma, lin=lin, X=ages, T=years
                    )
                    result = mean_absolute_error(QBongaarts, weights)
                    if np.isnan(result) or result is None:
                        print("NAN", alpha, beta, -gamma)
                        return float("inf")
                    else:
                        return result
                except (OverflowError, ValueError, ZeroDivisionError):
                    print(alpha, beta, gamma)
                    return float("inf")

            return objective

        return objective_builder_fixed_intervals(weights, ages, years, a_range, b_range, beta_range, gamma_range, lin)

    return create_objective

def median_ratio_in_age_range(K, L, projection_ages, age_range):
    """
    Compute the median ratio (L/K) in the specified age range for each year.

    Args:
        K (ndarray): Matrix of values indexed by projection_years and projection_ages.
        L (ndarray): Matrix of values indexed by projection_years and projection_ages.
        projection_ages (ndarray): Array of age indices (e.g., np.arange(0, 111)).
        age_range (tuple): Tuple specifying the age range (start_age, end_age).

    Returns:
        ndarray: Array of median ratios (L/K) for each year in the specified age range.
    """

    # Extract the start and end ages from the age range
    start_age, end_age = age_range

    # Find the column indices corresponding to the specified age range
    age_mask = (projection_ages >= start_age) & (projection_ages <= end_age)
    age_indices = np.where(age_mask)[0]

    # Compute the element-wise ratio L/K with handling of division by zero or very small values in K
    with np.errstate(divide='ignore', invalid='ignore'):
        ratio = np.divide(L, K)
        ratio[~np.isfinite(ratio)] = 0  # Replace inf and NaN with 0

    # Calculate the average ratio for each year (row-wise) over the specified age range
    median_ratio_per_year = np.median(ratio[:, age_indices], axis=1)

    return median_ratio_per_year

In [10]:
default_objective_builder = objective_builder_factory(
    a_range=(-0.1, -0.05),
    b_range=(0, 500),
    beta_range=(10e-2, 0.3),
    gamma_range=(10e-6, 10e-4),
    lin=False
)

comparative_objective_builders = [
    objective_builder_factory(
        a_range=(-0.5, -0.05),
        b_range=(0, 500),
        beta_range=(10e-2, 0.3),
        gamma_range=(10e-6, 10e-4),
        lin=False
    ),
    objective_builder_factory(
        a_range=(-0.1, -0.05),
        b_range=(0, 500),
        beta_range=(10e-2, 0.3),
        gamma_range=(10e-5, 10e-3),
        lin=False
    ),
    objective_builder_factory(
        a_range=(-0.1, -0.05),
        b_range=(0, 500),
        beta_range=(10e-3, 0.1),
        gamma_range=(10e-6, 10e-4),
        lin=False
    ),
    objective_builder_factory(
        a_range=(-0.1, -0.05),
        b_range=(0, 1000),
        beta_range=(10e-2, 0.3),
        gamma_range=(10e-6, 10e-4),
        lin=False
    )
]

In [ ]:
names = ["IRL_F"] #, "IRL_M", "UK_F", "UK_M"]

ages = np.arange(35, 65 + 1)
years = np.arange(2015, 2022 + 1)
projection_ages = np.arange(0, 110 + 1)
projection_years = np.arange(2015, 2050 + 1)
n_trials = 1000
age_ranges = [(40,50),(50,60)]

ratios = []
for name in names:
    default_adjusted_mortality_rates = bongaarts_study(
        name,
        ages=range(35, 65 + 1),
        years=years,
        projection_ages=projection_ages,
        projection_years=projection_years,
        objective_builder=default_objective_builder,
        n_trials=n_trials,
        plot=True,
        save=False,
    )
    plot_matrix(projection_ages, projection_years, default_adjusted_mortality_rates, title=f"Default Bongaarts {name}")

    for i, obj_builder in enumerate(comparative_objective_builders):
        mortality_rates = bongaarts_study(
            name,
            ages=range(35, 65 + 1),
            years=years,
            projection_ages=projection_ages,
            projection_years=projection_years,
            objective_builder=default_objective_builder,
            n_trials=n_trials,
            plot=False,
            save=False,
        )
        plot_matrix(projection_ages, projection_years, mortality_rates, title=f"Builder n°{i} Bongaarts {name}")

        for age_range in age_ranges:
            median_ratio = median_ratio_in_age_range(
                default_adjusted_mortality_rates,
                mortality_rates,
                projection_ages=projection_ages,
                age_range=age_range
            )
            for year_idx, year in enumerate(projection_years):
                ratios.append({
                    "Portfolio": name,
                    "Année": year,
                    "Déviation": median_ratio[year_idx]-1,
                    "Builder": i,
                    "Age Range": str(age_range)
                })

In [ ]:
from IPython.core.display import display, HTML

df = pd.DataFrame(ratios)

# Create a pivot table with Portfolio and Builder as index, years as columns, and Déviation as values
pivot_table = df.pivot_table(index=["Portfolio", "Age Range", "Builder"], columns="Année", values="Déviation")

pivot_table = pivot_table.applymap(lambda x: f"{'+' if x >= 0 else '-'}{abs(x * 100):.2f}%")

# Convert the pivot table to an HTML table
html_table = pivot_table.to_html()

# Convert the pivot table to a LaTeX table
latex_table = pivot_table.to_latex(escape=False)

# Save the LaTeX table to a file (optional)
with open("./tables/deviation_bongaarts_table.tex", "w") as file:
    file.write(latex_table)

display(HTML(html_table))